In [1]:
import pkg_resources

ls_libs = ["numpy", "pandas", "scipy", "scikit-learn"]
for n_lib in ls_libs:
    vak_ver = pkg_resources.get_distribution(n_lib).version
    print(f"{n_lib}: {vak_ver}")

numpy: 2.2.3
pandas: 2.2.3
scipy: 1.15.2
scikit-learn: 1.6.1


In [2]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import MinMaxScaler
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

### Q1. 

In [3]:
df = pd.read_csv("set_04_data.csv")
df.head(2)

,user,prod,gender,age_group,job,city,marital,prod_cat1,prod_cat2,prod_cat3,purchase
0,3451,P00375436,F,26-35,11,A,0,20,0,0,595
1,3829,P00003242,F,26-35,1,B,0,8,15,0,5872


In [7]:
df_u1665 = df.loc[df["user"] == 1665, ]
df_u1665["city"].value_counts()

city
B    14
D     1
Name: count, dtype: int64

In [8]:
df_u1665["city"].nunique()

2

In [ ]:
df_g_city_cnt = df.groupby("user")["city"].nunique().reset_index()
ser_user_city_plural = df_g_city_cnt.loc[df_g_city_cnt["city"] != 1, "user"]
ser_user_city_plural.head()

In [21]:
df_q1 = df.loc[~df["user"].isin(ser_user_city_plural), ].reset_index(drop = True)
len(df_q1)

54201

In [22]:
df_q1["user"].nunique()

3142

In [23]:
df_q1.head(1)

,user,prod,gender,age_group,job,city,marital,prod_cat1,prod_cat2,prod_cat3,purchase
0,5368,P00188442,F,26-35,17,C,0,5,7,0,8617


In [26]:
df_q1_c1 = df_q1.loc[(df_q1["job"] == 20) & (df_q1["gender"] == "M"), ] # condition
df_q1_c2 = df_q1_c1.loc[df_q1_c1["prod_cat1"] == 5, ] # target
len(df_q1_c1), len(df_q1_c2)

(1869, 552)

In [28]:
round(len(df_q1_c2) / len(df_q1_c1), 3)

0.295

### Q2. 

In [ ]:
df = pd.read_csv("set_04_data.csv")
df.head(2)

In [32]:
df_u1 = df.loc[df["user"] == 1, ["prod_cat1", "prod_cat2", "prod_cat3"]].reset_index(drop = True)
df_u1.head(2)

,prod_cat1,prod_cat2,prod_cat3
0,3,4,12
1,4,8,9


In [33]:
len(df_u1)

12

In [ ]:
df_u1.sort_values(df_u1.columns.to_list())

In [38]:
df_u1.drop_duplicates().shape

(9, 3)

In [43]:
df_u1["prod_cat1"] = df_u1["prod_cat1"].astype("str")
df_u1["prod_cat2"] = df_u1["prod_cat2"].astype("str")
df_u1["prod_cat3"] = df_u1["prod_cat3"].astype("str")
df_u1["prod_cat"] = df_u1["prod_cat1"] + "-" + df_u1["prod_cat2"] + "-" + df_u1["prod_cat3"]

In [44]:
df_u1.head(2)

,prod_cat1,prod_cat2,prod_cat3,prod_cat
0,3,4,12,3-4-12
1,4,8,9,4-8-9


In [45]:
df_u1["prod_cat"].nunique()

9

In [ ]:
df_q2 = df.loc[df["age_group"] == "26-35", ["user", "marital", "prod_cat1", "prod_cat2", "prod_cat3"]]
df_q2 = df_q2.reset_index(drop = True)
len(df_q2)

In [53]:
df_q2["prod_cat1"] = df_q2["prod_cat1"].astype("str")
df_q2["prod_cat2"] = df_q2["prod_cat2"].astype("str")
df_q2["prod_cat3"] = df_q2["prod_cat3"].astype("str")
df_q2["prod_cat" ] = df_q2["prod_cat1"] + "-" + df_q2["prod_cat2"] + "-" + df_q2["prod_cat3"]

In [ ]:
# df_q2_g = df_q2.groupby("user")["prod_cat"].nunique().reset_index()
df_q2_g = df_q2.groupby(["user", "marital"])["prod_cat"].nunique().reset_index()
df_q2_g.head()

In [ ]:
stat_m0 = df_q2_g.loc[df_q2_g["marital"] == 0, "prod_cat"].mean()
stat_m1 = df_q2_g.loc[df_q2_g["marital"] == 1, "prod_cat"].mean()
stat_m0, stat_m1

In [62]:
round(abs(stat_m0 - stat_m1), 2)

np.float64(0.03)

### Q3.

In [63]:
df = pd.read_csv("set_04_data.csv")
df.head(2)

,user,prod,gender,age_group,job,city,marital,prod_cat1,prod_cat2,prod_cat3,purchase
0,3451,P00375436,F,26-35,11,A,0,20,0,0,595
1,3829,P00003242,F,26-35,1,B,0,8,15,0,5872


In [ ]:
df_g_city_cnt = df.groupby("user")["city"].nunique().reset_index()
ser_user_city_plural = df_g_city_cnt.loc[df_g_city_cnt["city"] != 1, "user"]
df_s1 = df.loc[~df["user"].isin(ser_user_city_plural), ].reset_index(drop = True)

In [69]:
df_s2 = df_s1[["user", "gender", "age_group", "job", "city", "marital"]].drop_duplicates()

In [ ]:
df_s3 = df_s1.groupby("user")[["prod", "purchase"]].agg({"prod": "nunique", "purchase": "sum"}).reset_index()
df_s3.head(2)

In [ ]:
df_s4 = pd.merge(df_s2, df_s3, on = "user")
df_s4.head(2)

In [ ]:
df_s4["gender"] = df_s4["gender"].replace({"M": 1, "F": 0})

In [ ]:
# df_s4["age_group"].unique()
ser_u = df_s4["age_group"].sort_values().drop_duplicates()
ser_repl = pd.Series(range(len(ser_u)), index = ser_u)
ser_repl.to_dict()

In [ ]:
df_s4["age_group"] = df_s4["age_group"].replace(ser_repl) # Series 객체도 각 원소 치환 가능

In [78]:
df_s5 = df_s4.copy()

In [80]:
# df_s6 = pd.get_dummies(df_s5, columns = ["job", "city"]) # 시험버전
df_s6 = pd.get_dummies(df_s5, columns = ["job", "city"], dtype = "int")
df_s6 = df_s6.drop(columns = "user")
df_s6.head(1)

,gender,age_group,marital,prod,purchase,job_0,job_1,job_2,job_3,job_4,...,job_15,job_16,job_17,job_18,job_19,job_20,city_A,city_B,city_C,city_D
0,0,2,0,14,128695,0,0,0,0,0,...,0,0,1,0,0,0,0,0,1,0


In [81]:
arr_s7_nor = MinMaxScaler().fit_transform(df_s6)

In [83]:
# arr_s7_nor[:1, ]

In [ ]:
model_s8 = KMeans(n_clusters = 7, random_state = 123)
model_s8.fit(arr_s7_nor)

In [88]:
round(silhouette_score(arr_s7_nor, labels = model_s8.labels_), 2)

np.float64(0.17)